In [1]:
import tensorflow as tf
import datetime, os
import numpy as np
import pandas as pd
from PIL import Image

2023-03-27 16:26:58.489038: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
# Load the TensorBoard notebook extension
%load_ext tensorboard

In [3]:
def get_image(fp):
    """
    return the 32x32 greyscale array
    """
    img = Image.open(fp)
    return np.asarray(img)


## 1: Fully Connected Neural Network

In [10]:
# Feed forward network with:
    # 1024 neurors with tanh act
    # 512 neurons with sig act
    # 100 neurons with relu act
    # n neurons with softmax act
def create_model(n):
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.Dense(1024, input_shape=(1024,), activation='tanh', name='fc1'))
    model.add(tf.keras.layers.Dense(512, activation='sigmoid', name='fc2'))
    model.add(tf.keras.layers.Dense(100, activation='relu', name='fc3'))
    model.add(tf.keras.layers.Dense(n, activation='softmax', name='output'))
    return model

In [11]:
# Create train and test data from csvs in train and test folders
train_data = pd.read_csv('fairface_label_train.csv')[:10000]
test_data = pd.read_csv('fairface_label_train.csv')[:1000]

# Replace the file column with the actual images using get_image function
train_X = np.array([get_image(fp).flatten() for fp in train_data['file']])
test_X = np.array([get_image(fp).flatten() for fp in test_data['file']])

# Get the labels, 0 for Male and 1 for Female
train_Y = np.array([(0,1) if x == 'Female' else (1,0) for x in train_data['gender']])
test_Y = np.array([(0,1) if x == 'Female' else (1,0) for x in test_data['gender']])

# Min max normalize the data
max_val = np.max(train_X)
min_val = np.min(train_X)
train_X = (train_X - min_val) / (max_val - min_val)
test_X = (test_X - min_val) / (max_val - min_val)



In [12]:
print(type(test_Y))
print(test_Y.shape)

<class 'numpy.ndarray'>
(1000, 2)


### 1.1 Gender Classification

In [13]:
gender_model = create_model(2)
gender_model.compile(optimizer='sgd', loss='categorical_crossentropy', metrics=['accuracy'])

print(gender_model.summary())

logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)

# Train the model
gender_model.fit(x=train_X, 
                 y=train_Y, 
                 epochs=10, 
                 validation_data=(test_X, test_Y),
                 callbacks=[tensorboard_callback])

# Test the model

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 fc1 (Dense)                 (None, 1024)              1049600   
                                                                 
 fc2 (Dense)                 (None, 512)               524800    
                                                                 
 fc3 (Dense)                 (None, 100)               51300     
                                                                 
 output (Dense)              (None, 2)                 202       
                                                                 
Total params: 1,625,902
Trainable params: 1,625,902
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/10
313/313 [==============================] - 4s 9ms/step - loss: 0.6799 - accuracy: 0.5694 - val_loss: 0.6608 - val_accuracy: 0.6070
Epoch 2/10
313/313 [=========

In [14]:
%tensorboard --logdir logs